# OrientDB tutorial

## Prerequisites

### Documentation

You will find all documentation for :
* [OrientDB SQL reference](http://www.orientdb.com/docs/last/SQL-Functions.html)
* [Orientdb python client](http://orientdb.com/docs/last/PyOrient-Client.html#working-with-the-client)

## Import libraries

In [1]:
# !pip install pyorient

In [2]:
import pyorient

In [3]:
ROOT_PASSWORD = "root"
client = pyorient.OrientDB("localhost", 2424)
session_id = client.connect("root", ROOT_PASSWORD)

In [4]:
print(client.db_list())

{{'databases': {}}}


## I. Quick start

### Creating the database

**Q:** Create a database `gods` as a `GRAPH_DATABASE` in `MEMORY_STORAGE_TYPE`. 

We will use it to store relationships between Greek deities.

In [5]:
client.db_create("gods", pyorient.DB_TYPE_GRAPH, pyorient.STORAGE_TYPE_MEMORY)

**Q:** Connect your pyorient client to the `gods` database.

In [6]:
# Connect your pyorient client to the `gods` database.
client.db_open("gods", "root", ROOT_PASSWORD)

**Q:** You should now be able to launch OrientDB queries through the Python client with the [command()](http://orientdb.com/docs/last/PyOrient-Client-Command.html) function. 

You should think of OrientDB as a Graph-Document database for the following questions. Each vertex and edge will contain information on it inside a JSON document.

Create a new Vertex with content `{name: 'Zeus', symbol: 'thunder'}`. The [CREATE VERTEX : Create a vertex using JSON content](http://orientdb.com/docs/last/SQL-Create-Vertex.html) doc page should help you.

In [7]:
client.command("create class God extends V")

[11]

In [8]:
zeus = client.command("CREATE VERTEX God SET name = 'Zeus', symbol = 'thunder'")[0]

You have created a VERTEX in the previous question. The VERTEX is a [class](https://orientdb.com/docs/last/Tutorial-Classes.html) of OrientDB which defines a record that can be linked to others through EDGE instances.

You can find all VERTEX created in the database with a SQL command on the `V` table, like `SELECT * FROM V`. 

**Q:** Print all current vertices in `gods`, it should only have `Zeus` though for now.

In [9]:
result = client.command("SELECT FROM God")
for record in result:
    print(record)


{'@God':{'name': 'Zeus', 'symbol': 'thunder'},'version':1,'rid':'#33:0'}


**Q:** Create new vertices with content : 
```
{name:Héra, symbol:tiara}
{name:Poséidon, symbol:trident}
{name:Athena, symbol:helmet}
{name:Arès, symbol:weapons} 
```

In [10]:
hera = client.command("CREATE VERTEX God SET name = 'Héra', symbol = 'tiara'")[0]
poseidon = client.command("CREATE VERTEX God SET name = 'Poséidon', symbol = 'trident'")[0]
athena = client.command("CREATE VERTEX God SET name = 'Athena', symbol = 'helmet'")[0]
ares = client.command("CREATE VERTEX God SET name = 'Arès', symbol = 'weapons'")[0]

**Q:** Display all vertices with name = `Arès`

In [11]:
# Display all vertices with name = `Arès`
result = client.command("SELECT FROM God WHERE name = 'Arès'")
for record in result:
    print(record)

{'@God':{'name': 'Arès', 'symbol': 'weapons'},'version':1,'rid':'#37:0'}


**Q:** Create an EDGE from `Zeus` to `Poséidon` with the content `{kind: 'sibling'}

In [12]:
client.command("CREATE EDGE Sibling FROM (SELECT FROM God WHERE name = 'Zeus') TO (SELECT FROM God WHERE name = 'Poséidon') SET kind = 'sibling'")

**Q:** Redisplay all vertices, discuss.

In [13]:
result = client.command("SELECT FROM God")
for record in result:
    print(record)

# I can see all the vertex (including god's name and symbol) and edges (in or out) in the graph

{'@God':{'name': 'Zeus', 'symbol': 'thunder', 'out_Sibling': <pyorient.otypes.OrientBinaryObject object at 0x0000027ED548FEC8>},'version':2,'rid':'#33:0'}
{'@God':{'name': 'Héra', 'symbol': 'tiara'},'version':1,'rid':'#34:0'}
{'@God':{'name': 'Poséidon', 'symbol': 'trident', 'in_Sibling': <pyorient.otypes.OrientBinaryObject object at 0x0000027ED5492288>},'version':2,'rid':'#35:0'}
{'@God':{'name': 'Athena', 'symbol': 'helmet'},'version':1,'rid':'#36:0'}
{'@God':{'name': 'Arès', 'symbol': 'weapons'},'version':1,'rid':'#37:0'}


**Q:** Display all edges. They are contained in the class `E`

In [14]:
result = client.command("SELECT FROM E")
for record in result:
    print(record)

{'@Sibling':{'kind': 'sibling', 'out': <pyorient.otypes.OrientRecordLink object at 0x0000027ED5492C08>, 'in': <pyorient.otypes.OrientRecordLink object at 0x0000027ED5492D08>},'version':1,'rid':'#45:0'}


Two fields on vertices have appeared, containing the outgoing (out_) and incoming (in_) links.

At the edge level, two fields point to the original (out) and destination (in) vertices.

**Q:** Lets create some more edges :

* Zeus > Héra (sibling)
* Zeus > Arès (father)
* Zeus > Athena (father)
* Héra > Arès (mother)
* Héra > Zeus (sibling)
* Poséidon > Zeus (sibling)

_Hint 1 :_ check [the CREATE EDGE doc page](http://orientdb.com/docs/last/SQL-Create-Edge.html) to find an example for creating edges on vertices using subqueries so you can run queries to fetch the vertices before creating an edge in between.

_Hint 2 :_ after you have found the command to create edges between vertices with sub-queries, you should be well-versed enough in Python to create a list of all edges in the question, and loop the command on each element of the list to create all edges in one go =)

In [15]:
"""client.command("CREATE EDGE Sibling FROM (SELECT FROM God WHERE name = 'Zeus') TO (SELECT FROM God WHERE name = 'Héra')")
client.command("CREATE EDGE Father FROM (SELECT FROM God WHERE name = 'Zeus') TO (SELECT FROM God WHERE name = 'Arès')")
client.command("CREATE EDGE Father FROM (SELECT FROM God WHERE name = 'Zeus') TO (SELECT FROM God WHERE name = 'Athena')")
client.command("CREATE EDGE Mother FROM (SELECT FROM God WHERE name = 'Héra') TO (SELECT FROM God WHERE name = 'Arès')")
client.command("CREATE EDGE Sibling FROM (SELECT FROM God WHERE name = 'Héra') TO (SELECT FROM God WHERE name = 'Zeus')")
client.command("CREATE EDGE Sibling FROM (SELECT FROM God WHERE name = 'Poséidon') TO (SELECT FROM God WHERE name = 'Zeus')")"""

'client.command("CREATE EDGE Sibling FROM (SELECT FROM God WHERE name = \'Zeus\') TO (SELECT FROM God WHERE name = \'Héra\')")\nclient.command("CREATE EDGE Father FROM (SELECT FROM God WHERE name = \'Zeus\') TO (SELECT FROM God WHERE name = \'Arès\')")\nclient.command("CREATE EDGE Father FROM (SELECT FROM God WHERE name = \'Zeus\') TO (SELECT FROM God WHERE name = \'Athena\')")\nclient.command("CREATE EDGE Mother FROM (SELECT FROM God WHERE name = \'Héra\') TO (SELECT FROM God WHERE name = \'Arès\')")\nclient.command("CREATE EDGE Sibling FROM (SELECT FROM God WHERE name = \'Héra\') TO (SELECT FROM God WHERE name = \'Zeus\')")\nclient.command("CREATE EDGE Sibling FROM (SELECT FROM God WHERE name = \'Poséidon\') TO (SELECT FROM God WHERE name = \'Zeus\')")'

In [16]:
# the edges to create as a list of dictionaries
edges = [
    {"type": "Sibling", "from": "Zeus", "to": "Héra"},
    {"type": "Father", "from": "Zeus", "to": "Arès"},
    {"type": "Father", "from": "Zeus", "to": "Athena"},
    {"type": "Mother", "from": "Héra", "to": "Arès"},
    {"type": "Sibling", "from": "Héra", "to": "Zeus"},
    {"type": "Sibling", "from": "Poséidon", "to": "Zeus"}
]

for edge in edges:
    query = f"CREATE EDGE {edge['type']} FROM (SELECT FROM God WHERE name = '{edge['from']}') TO (SELECT FROM God WHERE name = '{edge['to']}')"
    client.command(query)


### Looking for data

**Q:** Using [out()](http://orientdb.com/docs/last/Tutorial-Working-with-graphs.html#querying-graphs) function, display all vertices connected and outgoing from Zeus.

You should use the EXPAND() special function to transform the vertex collection in the result-set by expanding it, making the results more readable.

In [17]:
result = client.command("SELECT expand(out()) FROM God WHERE name = 'Zeus'")
for record in result:
    print(record)

{'@God':{'name': 'Arès', 'symbol': 'weapons', 'in_Father': <pyorient.otypes.OrientBinaryObject object at 0x0000027ED5495588>, 'in_Mother': <pyorient.otypes.OrientBinaryObject object at 0x0000027ED5495848>},'version':3,'rid':'#37:0'}
{'@God':{'name': 'Athena', 'symbol': 'helmet', 'in_Father': <pyorient.otypes.OrientBinaryObject object at 0x0000027ED548FC88>},'version':2,'rid':'#36:0'}
{'@God':{'name': 'Poséidon', 'symbol': 'trident', 'in_Sibling': <pyorient.otypes.OrientBinaryObject object at 0x0000027ED5498188>, 'out_Sibling': <pyorient.otypes.OrientBinaryObject object at 0x0000027ED5498208>},'version':3,'rid':'#35:0'}
{'@God':{'name': 'Héra', 'symbol': 'tiara', 'in_Sibling': <pyorient.otypes.OrientBinaryObject object at 0x0000027ED5498408>, 'out_Mother': <pyorient.otypes.OrientBinaryObject object at 0x0000027ED5498488>, 'out_Sibling': <pyorient.otypes.OrientBinaryObject object at 0x0000027ED5498508>},'version':4,'rid':'#34:0'}


**Q:** Display all vertices which got a father (the vertices which are the destination of an arc whose kind attribute is father).

_Hint: You can notice that we use the field `in` the arc, and not the function `in()` which applies to vertices._

In [18]:
result = client.command("SELECT expand(in('Father')) FROM God")
for record in result:
    print(record)

{'@God':{'in_Sibling': <pyorient.otypes.OrientBinaryObject object at 0x0000027ED54988C8>, 'out_Father': <pyorient.otypes.OrientBinaryObject object at 0x0000027ED5498948>, 'out_Sibling': <pyorient.otypes.OrientBinaryObject object at 0x0000027ED54989C8>, 'name': 'Zeus', 'symbol': 'thunder'},'version':7,'rid':'#33:0'}
{'@God':{'in_Sibling': <pyorient.otypes.OrientBinaryObject object at 0x0000027ED5498C08>, 'out_Father': <pyorient.otypes.OrientBinaryObject object at 0x0000027ED5498C88>, 'out_Sibling': <pyorient.otypes.OrientBinaryObject object at 0x0000027ED5498D08>, 'name': 'Zeus', 'symbol': 'thunder'},'version':7,'rid':'#33:0'}


**Q:** As in SQL, the operator `in` used in a clause `where` allows to restrict the possible values with an embedded query _(where ... in (select ...))_. 

Display the mothers, by displaying the vertices where an outgoing arc is part of the arcs where kind is a mother.

In [19]:
query = """
SELECT * 
FROM God 
WHERE @rid IN (SELECT out FROM Mother)
"""
mothers = client.command(query)

for mother in mothers:
    print(f"Name: {mother.oRecordData.get('name')}")


Name: Héra


**Q:** Display the brothers and sisters of Zeus (the destination summits of an arc whose kind is sibling and whose original summit is Zeus).

In [20]:
result = client.command("SELECT expand(out('Sibling')) FROM God WHERE name = 'Zeus'")
for record in result:
    print(record)

{'@God':{'name': 'Poséidon', 'symbol': 'trident', 'in_Sibling': <pyorient.otypes.OrientBinaryObject object at 0x0000027ED549A688>, 'out_Sibling': <pyorient.otypes.OrientBinaryObject object at 0x0000027ED549A708>},'version':3,'rid':'#35:0'}
{'@God':{'name': 'Héra', 'symbol': 'tiara', 'in_Sibling': <pyorient.otypes.OrientBinaryObject object at 0x0000027ED549A908>, 'out_Mother': <pyorient.otypes.OrientBinaryObject object at 0x0000027ED549A988>, 'out_Sibling': <pyorient.otypes.OrientBinaryObject object at 0x0000027ED549AA08>},'version':4,'rid':'#34:0'}


## Modeling a Product Recommendation System

You are currently modeling the data of a product recommendation system with OrientDB.

The main purpose of such a system is to answer the question "which products were purchased by their people who purchased product X? »

Purchased products have only one name field. They are purchased by people who have a nickname.

When a person buys a product, the date of purchase is stored. 

Instead of working with "anonymous" vertices and arcs, you will use classes. The `create class` command allows you to create custom classes.

The vertex classes must extend V, the arc classes must extend E.

**Q:** Create an `eCommerce` database, and the necessary classes to model the system.

PS : you can view all classes in the database with :

```python
for name in client.command("SELECT name FROM (SELECT expand(classes) FROM metadata:schema)"):
    print(name)
```

In [21]:
# create eCommerce db
client.db_create("eCommerce", pyorient.DB_TYPE_GRAPH, pyorient.STORAGE_TYPE_MEMORY)

In [22]:
for name in client.command("SELECT name FROM (SELECT expand(classes) FROM metadata:schema)"):
    print(name)

{{'name': 'OTriggered'},'version':0,'rid':'#-2:0'}
{{'name': 'OUser'},'version':0,'rid':'#-2:1'}
{{'name': 'ORole'},'version':0,'rid':'#-2:2'}
{{'name': 'E'},'version':0,'rid':'#-2:3'}
{{'name': 'ORestricted'},'version':0,'rid':'#-2:4'}
{{'name': 'OIdentity'},'version':0,'rid':'#-2:5'}
{{'name': 'OFunction'},'version':0,'rid':'#-2:6'}
{{'name': 'OSequence'},'version':0,'rid':'#-2:7'}
{{'name': 'V'},'version':0,'rid':'#-2:8'}
{{'name': 'OSchedule'},'version':0,'rid':'#-2:9'}


**Q:** Create the following products: `spaghetti`, `bolognese sauce`, `cheese`, `apple`.

In [23]:
# Create the following products: `spaghetti`, `bolognese sauce`, `cheese`, `apple`.
client.command("CREATE CLASS Product extends V")
client.command("CREATE VERTEX Product SET name = 'spaghetti'")
client.command("CREATE VERTEX Product SET name = 'bolognese sauce'")
client.command("CREATE VERTEX Product SET name = 'cheese'")
client.command("CREATE VERTEX Product SET name = 'apple'")

**Q:** Create the following people: `peter`, `meredith`.

In [24]:
client.command("CREATE CLASS Person extends V")
client.command("CREATE VERTEX Person SET name = 'peter'")
client.command("CREATE VERTEX Person SET name = 'meredith'")

**Q:** Create the following purchases: 
- peter > spaghetti + cheese on 20/01/2016 
- meredith > cheese + apple + bolognese sauce on 22/01/2016
- peter > spaghetti + bolognese sauce on 27/01/2016


In [25]:

# - peter > spaghetti + cheese on 20/01/2016
client.command("CREATE EDGE Bought FROM (SELECT FROM Person WHERE name = 'peter') TO (SELECT FROM Product WHERE name = 'spaghetti') SET date = '20/01/2016'")
client.command("CREATE EDGE Bought FROM (SELECT FROM Person WHERE name = 'peter') TO (SELECT FROM Product WHERE name = 'cheese') SET date = '20/01/2016'")
# - meredith > cheese + apple + bolognese sauce on 22/01/2016
client.command("CREATE EDGE Bought FROM (SELECT FROM Person WHERE name = 'meredith') TO (SELECT FROM Product WHERE name = 'cheese') SET date = '22/01/2016'")
client.command("CREATE EDGE Bought FROM (SELECT FROM Person WHERE name = 'meredith') TO (SELECT FROM Product WHERE name = 'apple') SET date = '22/01/2016'")
client.command("CREATE EDGE Bought FROM (SELECT FROM Person WHERE name = 'meredith') TO (SELECT FROM Product WHERE name = 'bolognese sauce') SET date = '22/01/2016'")
# - peter > spaghetti + bolognese sauce on 27/01/2016
client.command("CREATE EDGE Bought FROM (SELECT FROM Person WHERE name = 'peter') TO (SELECT FROM Product WHERE name = 'spaghetti') SET date = '27/01/2016'")
client.command("CREATE EDGE Bought FROM (SELECT FROM Person WHERE name = 'peter') TO (SELECT FROM Product WHERE name = 'bolognese sauce') SET date = '27/01/2016'")

**Q:** Who bought Bolognese sauce?

In [26]:
result=client.command("SELECT expand(in('Bought')) FROM Product WHERE name = 'bolognese sauce'")
for record in result:
    print(record)

{'@Person':{'out_Bought': <pyorient.otypes.OrientBinaryObject object at 0x0000027ED54AF0C8>, 'name': 'meredith'},'version':4,'rid':'#46:0'}
{'@Person':{'out_Bought': <pyorient.otypes.OrientBinaryObject object at 0x0000027ED54AF2C8>, 'name': 'peter'},'version':5,'rid':'#45:0'}


**Q:** It is possible to link the `out` and `in` navigation functions. What products are purchased with Bolognese sauce? 

In [28]:
result = client.command("SELECT expand(in().out()) FROM Product WHERE name = 'bolognese sauce'")
for record in result:
    print(record)

{'@Product':{'in_Bought': <pyorient.otypes.OrientBinaryObject object at 0x0000027ED548BF08>, 'name': 'cheese'},'version':3,'rid':'#35:0'}
{'@Product':{'name': 'apple', 'in_Bought': <pyorient.otypes.OrientBinaryObject object at 0x0000027ED548BC48>},'version':2,'rid':'#36:0'}
{'@Product':{'in_Bought': <pyorient.otypes.OrientBinaryObject object at 0x0000027ED548B248>, 'name': 'bolognese sauce'},'version':3,'rid':'#34:0'}
{'@Product':{'in_Bought': <pyorient.otypes.OrientBinaryObject object at 0x0000027ED5EF1BC8>, 'name': 'spaghetti'},'version':3,'rid':'#33:0'}
{'@Product':{'in_Bought': <pyorient.otypes.OrientBinaryObject object at 0x0000027ED5EF1AC8>, 'name': 'cheese'},'version':3,'rid':'#35:0'}
{'@Product':{'in_Bought': <pyorient.otypes.OrientBinaryObject object at 0x0000027ED5EF8508>, 'name': 'spaghetti'},'version':3,'rid':'#33:0'}
{'@Product':{'in_Bought': <pyorient.otypes.OrientBinaryObject object at 0x0000027ED5EF8148>, 'name': 'bolognese sauce'},'version':3,'rid':'#34:0'}


## Postquisites

Since we create databases in memory, they get destroyed on server shutdown.